In [ ]:
# Cell 1: Imports, counters & max‑requests guard
import requests
import pandas as pd
import time
import traceback
from random import sample
from typing import Dict, List

REQUEST_COUNT = 0
MAX_REQUESTS  = 1000

class MaxRequestsReached(Exception):
    """Raised when REQUEST_COUNT hits MAX_REQUESTS."""
    pass


In [ ]:
# Cell 2: API keys, limits & rotation helpers
API_KEYS = []
KEY_LIMIT = 500
key_index = 0
key_usage = [0] * len(API_KEYS)

API_HOST   = "imdb236.p.rapidapi.com"
SEARCH_URL = f"https://{API_HOST}/imdb/search"

def get_headers() -> Dict[str, str]:
    return {
        "x-rapidapi-key": API_KEYS[key_index],
        "x-rapidapi-host": API_HOST
    }

def switch_key():
    global key_index
    start = key_index
    for offset in range(1, len(API_KEYS)):
        nxt = (start + offset) % len(API_KEYS)
        if key_usage[nxt] < KEY_LIMIT:
            key_index = nxt
            print(f"\n↺ Switched to key #{key_index} ({key_usage[key_index]}/{KEY_LIMIT})")
            return
    raise RuntimeError("All API keys exhausted.")


In [ ]:
# Cell 3: Single‑request fetcher with guard, key rotation & retry
def fetch_n_movies(params: Dict, n: int, delay: float = 0.3) -> List[Dict]:
    """
    Fetch up to `n` movies in one request (rows=n),
    auto‑rotating keys, retrying on failure, and
    stopping if MAX_REQUESTS is reached.
    """
    global REQUEST_COUNT, key_usage, key_index

    # guard
    if REQUEST_COUNT >= MAX_REQUESTS:
        raise MaxRequestsReached()

    p = params.copy()
    p.update({"type": "movie", "rows": n})

    while True:
        # rotate if needed
        if key_usage[key_index] >= KEY_LIMIT:
            switch_key()

        # count this request
        REQUEST_COUNT += 1
        key_usage[key_index] += 1
        if REQUEST_COUNT > MAX_REQUESTS:
            raise MaxRequestsReached()
        print(f"Req #{REQUEST_COUNT} | Key {key_index} ({key_usage[key_index]}/{KEY_LIMIT})", end="\r")

        try:
            resp = requests.get(SEARCH_URL, headers=get_headers(), params=p, timeout=10)
            resp.raise_for_status()
            data = resp.json()
            return data.get("results") or data.get("data") or []
        except MaxRequestsReached:
            raise
        except Exception as e:
            print(f"\nRequest error on key {key_index}: {e}. Switching key...")
            switch_key()
            time.sleep(delay)
            continue


In [ ]:
# Cell 4: Define sampling buckets & parameters
genres        = ["Drama", "Comedy", "Action", "Documentary", "Thriller", "Animation"]
rating_ranges = [(0, 5.0), (5.1, 7.0), (7.1, 10.0)]
vote_ranges   = [
    (2_000,      10_000),
    (10_001,     100_000),
    (100_001,   1_000_000),
    (1_000_001, 3_500_000)
]
regions       =  ["BR"]

N_PER_BUCKET = 50
START_YEAR   = 2020

# We’ll loop over both sort directions:
sort_orders = ["ASC", "DESC"]


In [ ]:
# Cell 5: Run sampling loop (ASC & DESC), with MAX_REQUESTS guard & partial‑save
all_samples: List[Dict] = []
seen_ids = set()

try:
    for genre in genres:
        for rmin, rmax in rating_ranges:
            for vmin, vmax in vote_ranges:
                for country in regions:
                    base_q = {
                        "startYearFrom": START_YEAR,
                        "averageRatingFrom": rmin,
                        "averageRatingTo": rmax,
                        "numVotesFrom": vmin,
                        "numVotesTo": vmax,
                        "sortField": "id",
                    }
                    if genre:
                        base_q["genre"] = genre
                    if country:
                        base_q["countriesOfOrigin"] = country

                    for order in sort_orders:
                        q = base_q.copy()
                        q["sortOrder"] = order

                        movies = fetch_n_movies(q, N_PER_BUCKET)
                        # filter out duplicates
                        new = [m for m in movies if m["id"] not in seen_ids]
                        # if more than needed, random‑sample down
                        chosen = new if len(new) <= N_PER_BUCKET else sample(new, N_PER_BUCKET)
                        for m in chosen:
                            seen_ids.add(m["id"])
                            all_samples.append(m)

    print(f"\nDone: {len(all_samples)} movies, {REQUEST_COUNT} requests used")

except MaxRequestsReached:
    print(f"\n🚫 Reached MAX_REQUESTS ({MAX_REQUESTS}). Stopping early.")

except Exception as e:
    print(f"\n⚠️ Error during sampling: {e}")
    traceback.print_exc()

finally:
    # always save progress
    try:
        pd.json_normalize(all_samples).to_excel("movies_sample_partial.xlsx", index=False)
        print("💾 Progress saved to movies_sample_partial.xlsx")
    except Exception as e2:
        print(f"❌ Failed saving progress: {e2}")


Req #1 | Key 0 (1/500)
Request error on key 0: 429 Client Error: Too Many Requests for url: https://imdb236.p.rapidapi.com/imdb/search?startYearFrom=2020&averageRatingFrom=0&averageRatingTo=5.0&numVotesFrom=2000&numVotesTo=10000&sortField=id&genre=Drama&countriesOfOrigin=BR&sortOrder=ASC&type=movie&rows=50. Switching key...

↺ Switched to key #1 (0/500)

Done: 26 movies, 145 requests used
💾 Progress saved to movies_sample_partial.xlsx


In [ ]:
# Cell 6: Assemble into DataFrame & inspect
df = pd.json_normalize(all_samples)
print(df.shape)
df.head()


(26, 26)


,id,url,primaryTitle,originalTitle,type,description,primaryImage,trailer,contentRating,isAdult,...,countriesOfOrigin,externalLinks,spokenLanguages,filmingLocations,productionCompanies,budget,grossWorldwide,averageRating,numVotes,metascore
0,tt26787296,https://www.imdb.com/title/tt26787296/,Burning Betrayal,O Lado Bom de ser Traída,movie,Babi discovers a betrayal by her long-term par...,https://m.media-amazon.com/images/M/MV5BOGVmOD...,https://www.youtube.com/watch?v=fYQpOLrQ-BU,TV-MA,False,...,[BR],None,[pt],"[São Paulo, São Paulo, Brazil]","[{'id': 'co0307800', 'name': 'Glaz Entertainme...",NaN,NaN,4.2,4084,NaN
1,tt10395866,https://www.imdb.com/title/tt10395866/,Executive Order,Medida Provisória,movie,"In a dystopian near future in Brazil, an autho...",https://m.media-amazon.com/images/M/MV5BZGM5Mj...,https://www.youtube.com/watch?v=ILEcUPviEyM,None,False,...,[BR],[https://www.instagram.com/medidaprovisoriaofi...,[pt],"[Rio de Janeiro, Rio de Janeiro, Brazil]","[{'id': 'co0087755', 'name': 'Lereby Productio...",NaN,8419.0,6.6,2587,NaN
2,tt10605812,https://www.imdb.com/title/tt10605812/,The Girl Who Killed Her Parents,A Menina que Matou os Pais,movie,A psychological thriller surrounding the real ...,https://m.media-amazon.com/images/M/MV5BODg1ZD...,None,None,False,...,[BR],None,[pt],None,"[{'id': 'co0753951', 'name': 'Santa Rita Filme...",8000000.0,NaN,5.9,3958,NaN
3,tt26304178,https://www.imdb.com/title/tt26304178/,O'Dessa,O'Dessa,movie,A farm girl in search to recover a cherished f...,https://m.media-amazon.com/images/M/MV5BZTAyMj...,https://www.youtube.com/watch?v=-YDQR6NUNYQ,PG-13,False,...,"[BR, US]",None,[en],"[Zagreb, Croatia]","[{'id': 'co0028932', 'name': 'Searchlight Pict...",NaN,NaN,5.1,2644,41.0
4,tt15132028,https://www.imdb.com/title/tt15132028/,Private Desert,Deserto Particular,movie,"Daniel is an exemplary policeman, but he ends ...",https://m.media-amazon.com/images/M/MV5BY2NiOG...,https://www.youtube.com/watch?v=TOsfXrtgzno,Unrated,False,...,"[BR, PT]",[https://grafoaudiovisual.com/en/cinema/desert...,[pt],None,"[{'id': 'co0180703', 'name': 'Pandora Filmes'}...",NaN,34414.0,7.0,2026,66.0


In [ ]:
# Cell 7: Final export to Excel with fallback
output_file  = "movies_sample.xlsx"
partial_file = "movies_sample_partial.xlsx"

try:
    df.to_excel(output_file, index=False)
    print(f"✅ Exported full dataset to {output_file}")
except Exception as e:
    print(f"⚠️ Export error: {e}")
    traceback.print_exc()
    df.to_excel(partial_file, index=False)
    print(f"💾 Partial export to {partial_file}")


✅ Exported full dataset to movies_sample.xlsx


In [ ]:
# Cell 8: Fetch IMDb list endpoints and merge into main DataFrame

# List of static endpoints to pull
endpoints = [
    "/imdb/top250-movies",
    "/imdb/top-box-office",
    "/imdb/most-popular-movies",
    "/imdb/lowest-rated-movies"
]

dfs_endpoints = []
for ep in endpoints:
    # Rotate key if needed
    if key_usage[key_index] >= KEY_LIMIT:
        switch_key()
    # Count request
    REQUEST_COUNT += 1
    key_usage[key_index] += 1
    print(f"Fetching {ep} (Req #{REQUEST_COUNT})", end="\r")

    url = f"https://{API_HOST}{ep}"
    resp = requests.get(url, headers=get_headers(), timeout=10)
    resp.raise_for_status()
    data = resp.json()

    # Convert to DataFrame
    df_temp = pd.DataFrame(data)
    dfs_endpoints.append(df_temp)

# Combine endpoint DataFrames
df_endpoints = pd.concat(dfs_endpoints, ignore_index=True)

# Append to main df and drop duplicates by 'id'
df = pd.concat([df, df_endpoints], ignore_index=True)
df.drop_duplicates(subset=["id"], keep="first", inplace=True)

print(f"\nAfter appending endpoints: {df.shape[0]} total unique movies")



After appending endpoints: 467 total unique movies


In [ ]:
# Cell 7: Final export to Excel with fallback
output_file  = "movies_sample_total.xlsx"
partial_file = "movies_sample_partial_total.xlsx"

try:
    df.to_excel(output_file, index=False)
    print(f"✅ Exported full dataset to {output_file}")
except Exception as e:
    print(f"⚠️ Export error: {e}")
    traceback.print_exc()
    df.to_excel(partial_file, index=False)
    print(f"💾 Partial export to {partial_file}")


✅ Exported full dataset to movies_sample_total.xlsx
